## Detecting Ships in Satellite Images from the Airbus Dataset using Mask RCNN

Joel Pearson

---

Fristly, we will need to clone the [Mask RCNN](https://github.com/matterport/Mask_RCNN) model from Github. We will also need to download [COCO](https://cocodataset.org/#home) pre-trained weights. 

In [ ]:
import git
import requests
import os

# clone git repo into folder 'Mask_RCNN'
try:
    git.Repo.clone_from('https://github.com/matterport/Mask_RCNN', 'Mask_RCNN')
except git.GitCommandError:
    print("Destination path 'Mask_RCNN' likely already exists, and is not empty.")
except:
    print('Something else went wrong.')
    
# download coco weights
coco_url = 'https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5'
filename = 'Mask_RCNN/' + coco_url.split('/')[-1]
if os.path.exists(filename):
    print('Path to weights already exists. Continuing...')
    pass
else:
    r = requests.get(coco_url, allow_redirects=True)
    open(filename, 'wb').write(r.content)

### Prepare the Model Configuration Parameters

In [ ]:
from Mask_RCNN.mrcnn.config import Config

class ModelConfig(Config):
    NAME = 'airbus_config'
    
    GPU_COUNT = 1  # number of GPUs to use
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2  # ship and background
    
    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 768
    
    STEPS_PER_EPOCH = 300 
    VALIDATION_STEPS = 50
    
    BACKBONE = 'resnet50'
    RPN_ANCHOR_SCALES = (8, 16, 32, 64)
    TRAIN_ROIS_PER_IMAGE = 64
    MAX_GT_INSTANCES = 14
    DETECTION_MAX_INSTANCES = 10
    SAVE_BEST_ONLY = True
    
    # Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.95

    # Non-maximum suppression threshold for detection
    # Keep it small to merge overlapping ROIs 
    DETECTION_NMS_THRESHOLD = 0.0
    
    ## balance out losses
    LOSS_WEIGHTS = {
        "rpn_class_loss": 30.0,
        "rpn_bbox_loss": 0.8,
        "mrcnn_class_loss": 6.0,
        "mrcnn_bbox_loss": 1.0,
        "mrcnn_mask_loss": 1.2
    }

### Build the Mask R-CNN Model Architecture

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append(os.path.join('Mask_RCNN'))
import mrcnn.model

model = mrcnn.model.MaskRCNN(mode='training', 
                       config=ModelConfig(), 
                       model_dir='logs')
model.keras_model.summary()

### Load the Model Weights

In [ ]:
model.load_weights(filepath='Mask_RCNN/mask_rcnn_coco.h5', by_name=True)